This is a cover letter builder using agentic ai


In [2]:
#imports

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from pydantic import BaseModel
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [3]:
load_dotenv(override=True)


True

In [4]:
openai = OpenAI()

In [5]:
creator_model = "gpt-4o"
evaluator_model = "o4-mini"

In [6]:
with open("../me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

with open("../me/cover_letter_template.txt", "r", encoding="utf-8") as f:
    cover_letter_template = f.read()
    

In [7]:
reader = PdfReader("../me/resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [8]:
name = "Sviatoslav Rutkovskyi"


In [9]:
system_prompt = f"""You are a proffesional cover letter writer, and your job is to write a cover letter for {name}, highlighting {name}'s skills, experience, and achievements. 
particularly questions related to {name}'s career, background, skills and experience. 
Your responsibility is to represent {name} in the letter as faithfully as possible. 
You are given a summary of {name}'s background and Resume which you can use in the cover letter. 
You are given an example of a cover letter from {name}. Try and use a similar language and style. Do NOT include the placeholder information in the cover letter. 
Be professional and engaging, uing the tone and style suitable for a cover letter.
Do not make up any information, and only use the information provided.
Don't be too verbose, and use a 3 paragraph format.
Respond with a cover letter and nothing else.
Do not include the address or contact information. 
You will be given a job description, and you will need to tailor the cover letter to the job description.
You will be evaluated, and if evalutor decides that your cover letter is not up to standart, you will be given your previus cover letters and feedback on them. 
You have to listen to the feedback, and improve your cover letter accordingly to the feedback.
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n ## Cover Letter Template:\n{cover_letter_template}\n\n
"""



In [10]:

updated_system_prompt = system_prompt

In [11]:
evaluator_system_prompt = f"""
You are a professional evaluator that decides whether a cover letter is acceptable. 
You are provided with {name}'s summary and resume, an example of a cover letter from {name}, the job description, and the cover letter. 
Your task is to evaluate the cover letter, and reply with whether it is acceptable and your feedback. 
You need to ensure if the cover letter is professional, engaging, and tailored to the job description. 
You need to ensure if the cover letter was likely made by AI, and if it was made by AI, deny it, and provide feedback. Do not allow AI generated cover letters.
You need to ensure that the cover letter has a strong and engaging opening paragraph. 
You need to ensure that the cover letter is concise and uses the standard 3 paragraph format.
Here's the information:
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n## Cover Letter Template:\n{cover_letter_template}\n\n
With this context, please evaluate the cover letter, replying with whether the cover letter is acceptable and your feedback.
"""

In [12]:
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [13]:
def evaluator_cover_letter(job_post, cover_letter):
    return f"""
    Here's the job posting presented by the user: \n\n{job_post}\n\n
    Here's the cover letter generated by the agent: \n\n{cover_letter}\n\n
    Please evaluate the response, replying with whether it is acceptable and your extensive feedback.
    """

In [14]:
def update_system_prompt(cover_letter, feedback):
        global updated_system_prompt
        updated_system_prompt = updated_system_prompt + f"""
        \n\n## Previous cover letter rejected\nYou just tried to create a cover letter, but the quality control rejected your cover letter\n
        ## Your attempted cover letter:\n{cover_letter}\n\n
        ## Reason for rejection:\n{feedback}\n\n
        """
        return updated_system_prompt;

In [15]:
def evaluate(job_post, cover_letter) -> Evaluation:
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_cover_letter(job_post, cover_letter)},
        {"role": "user", "content": "Reply ONLY in valid JSON: {\"is_acceptable\": true/false, \"feedback\": \"...\"}"}
    ]
    response = openai.responses.parse(
        model=evaluator_model,
        reasoning = {"effort":"high"},
        input=messages,
        text_format = Evaluation,
        )
    return response.output_parsed

In [16]:
def run(prompt, job_posting):
    messages = [{"role": "system", "content": prompt}] + [{"role": "user", "content": job_posting}]
    response = openai.chat.completions.create(model=creator_model, messages=messages)
    return response.choices[0].message.content

In [17]:

def scrape_webpage_simple(url):
    try:
        # Set headers to mimic a real browser
        headers = {
            'User-Agent': 'Mozilla/5.0'
        }
        
        # Make the request
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Parse the HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Get all text content
        return soup.get_text()
        
        
    except requests.RequestException:
        return 'error'
    except Exception:
        return 'error'


In [18]:
def requestLetter(job_posting, history):
    global updated_system_prompt
    page = scrape_webpage_simple(job_posting)
    print(page)
    if page == 'error':
        print("Failed to scrape job posting")
    else:
        job_posting = page

    cover_letter = run(system_prompt, job_posting)

    eval_counter = 0
    while eval_counter < 10:
        evaluation = evaluate(job_posting, cover_letter)
        if evaluation.is_acceptable:
            print("Passed evaluation - returning reply")
            display(Markdown(f"## Cover Letter:\n{cover_letter}"));
            display(Markdown(f"## Feedback:\n{evaluation.feedback}"))
            display(Markdown(f"## Updated system prompt:\n{updated_system_prompt}"))
            updated_system_prompt = system_prompt;
            return cover_letter
        else:
            eval_counter += 1
            print("Failed evaluation - retrying")
            display(Markdown(f"## Cover Letter:\n{cover_letter}"))
            display(Markdown(f"## Feedback:\n{evaluation.feedback}"))
            cover_letter = run(update_system_prompt(cover_letter, evaluation.feedback), job_posting)
    print("Failed evaluation - returning reply")
    return "Unable to generate cover letter" +"\n" + evaluation.feedback    
    

In [19]:
gr.ChatInterface(requestLetter, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.











Full Stack Engineer Co-Op Internship @ Stably







 







      You need to enable JavaScript to run this app.
    






 
 

Failed evaluation - retrying


## Cover Letter:
Dear Hiring Team,

I am thrilled to apply for the Full Stack Engineer Co-Op Internship at Stably. As a recent graduate from the University of Washington, Tacoma, with a Bachelor of Science in Computer Science and a GPA of 3.9, I am eager to leverage my skills and knowledge in software development to contribute to your innovative team. My technical proficiency and problem-solving abilities, combined with my passion for creating positive change, align perfectly with the role at Stably.

Throughout my academic career, I have developed full-stack applications, demonstrating my expertise in both front-end and back-end development. Notably, I led a team to create a comprehensive bookkeeping web app with a React/Next.js frontend and a robust Node.js, Express, and PostgreSQL backend. This experience honed my skills in API design, Docker containerization, and collaboration in team settings. Additionally, my role as a sergeant in the US Army Reserves has equipped me with leadership and teamwork skills, enabling me to thrive in high-pressure environments and collaborate effectively.

Stably's mission to enhance financial systems resonates deeply with my goal of making a meaningful impact through technology. I admire your commitment to innovation in finance and am eager to contribute to your projects with dedication and enthusiasm. I am confident that my technical skills, coupled with my commitment to excellence, make me an ideal candidate for this internship. I look forward to the opportunity to contribute to Stably’s success and to grow professionally with your team.

Thank you for considering my application.

Sincerely,

Sviatoslav Rutkovskyi

## Feedback:
This cover letter reads like a generic, AI-generated template rather than a tailored message to Stably. The opening is polite but lacks a strong hook about why you specifically want to work at Stably—mentioning their stablecoin technology or blockchain mission would show you’ve done your research. Your body paragraph highlights relevant projects, but you should connect those experiences directly to the skills and responsibilities listed in the internship posting (e.g. emphasize any front-end frameworks or DevOps tools Stably uses). The closing is courteous but could include a clear call to action and reiterate your enthusiasm for this particular role. To improve, personalize the letter by referencing Stably’s products or culture, add an engaging story or outcome from your past work, and demonstrate exactly how your background will advance their goals.

Failed evaluation - retrying


...

## Feedback:
This cover letter reads as a polished but generic AI-generated response rather than a personalized message to Stably. While it follows a three-paragraph structure and mentions relevant projects and leadership experience, the opening is standard (“I am writing to express my interest...”), and the language is formulaic. To improve: 

1. Strengthen the opening with a specific hook – perhaps a brief anecdote or direct reference to Stably’s recent product launch or technical challenge that resonated with you. 
2. Inject your genuine voice and passion: explain why blockchain and stablecoins matter to you personally and how your background uniquely positions you to contribute. 
3. Tie your skills more directly to Stably’s advertised tech stack or responsibilities (e.g., mention familiarity with their CI/CD pipeline, smart contract languages, or specific frameworks). 
4. Close with an active next step (“I would welcome the opportunity to discuss how my experience in X and Y can help Stably accelerate its roadmap”).

Personalizing these elements will make the letter more engaging, less templated, and clearly written by you rather than an AI.

Passed evaluation - returning reply


## Cover Letter:
Dear Hiring Team,

I am enthusiastic about applying for the Full Stack Engineer Co-Op Internship at Stably. As a diligent Computer Science graduate from the University of Washington, Tacoma, with a 3.9 GPA, I have developed a strong foundation in software development and am eager to apply my skills to your innovative blockchain and stablecoin initiatives.

My academic journey includes leading a team to create a bookkeeping web app with a React/Next.js frontend and a Node.js, Express, and PostgreSQL backend. This experience refined my API design skills and Docker containerization knowledge while fostering adaptability and teamwork. Additionally, my service as a sergeant in the US Army Reserves has fortified my leadership and communication abilities, preparing me to excel in Stably's dynamic environment.

I am particularly drawn to Stably’s commitment to financial inclusion through blockchain. Your work with stablecoins, notably Stably USD, aligns with my aspiration to create impactful technical solutions in finance. I am excited about the opportunity to contribute to your groundbreaking projects and support Stably’s mission to transform financial systems.

Thank you for considering my application.

Sincerely,

Sviatoslav Rutkovskyi

## Feedback:
This cover letter is professional, concise, and follows the three-paragraph format. It opens with genuine enthusiasm, cites your 3.9 GPA and relevant software development background, and clearly ties your React/Next.js, Node.js/Express, PostgreSQL, and Docker experience to Stably’s blockchain and stablecoin work. Your mention of leadership in the US Army Reserves adds a nice personal touch and underscores your ability to perform under pressure. To make it even stronger, you could: • Reference any coursework or personal projects related to blockchain or cryptography to reinforce your domain interest. • Add a brief, specific sentence about how you’d contribute to one of Stably’s products (e.g., improving performance or building new features for Stably USD). • Include a call to action in your closing (for example, “I welcome the opportunity to discuss how my skills can help drive Stably’s mission”). Overall, this letter is well-tailored and ready to submit.

## Updated system prompt:
You are a proffesional cover letter writer, and your job is to write a cover letter for Sviatoslav Rutkovskyi, highlighting Sviatoslav Rutkovskyi's skills, experience, and achievements. 
particularly questions related to Sviatoslav Rutkovskyi's career, background, skills and experience. 
Your responsibility is to represent Sviatoslav Rutkovskyi in the letter as faithfully as possible. 
You are given a summary of Sviatoslav Rutkovskyi's background and Resume which you can use in the cover letter. 
You are given an example of a cover letter from Sviatoslav Rutkovskyi. Try and use a similar language and style. Do NOT include the placeholder information in the cover letter. 
Be professional and engaging, uing the tone and style suitable for a cover letter.
Do not make up any information, and only use the information provided.
Don't be too verbose, and use a 3 paragraph format.
Respond with a cover letter and nothing else.
Do not include the address or contact information. 
You will be given a job description, and you will need to tailor the cover letter to the job description.
You will be evaluated, and if evalutor decides that your cover letter is not up to standart, you will be given your previus cover letters and feedback on them. 
You have to listen to the feedback, and improve your cover letter accordingly to the feedback.


## Summary:
My name is Sviatoslav Rutkovskyi, and I am a recent graduate from university of Washington Tacoma, with a Bachelor of Science in Computer Science, and 3.9 cumulative GPA. I also have a minor in Mathematics. I am an aspiring software developer, and want to use my skills to make a positive change in the world. 
My hobbies include playing guitar, hiking, and skiing. I bought skiing gear in April, and I'm excited to give them a test drive once the season arrives.

My interest in coding started in highschool, when I would write simple python code in my free time. During my running start at Bellevue college, I decided to keep pursing my inteterest in programming, and took some introductory computer science classes, and worked on getting my prerequisites completed for a Bachelor of Science in Computer Science. After finishing running start, I quickly joined the army reserves, and completed the bootcamp. 
After returning from the army bootcamp, I started preparing for the next step in my education, while taking online courses on web development and PHP on the side. I decided to go to the University of Washington, Tacoma campus to continue my higher education. At the same time, I was still working part time at home depot, while maintaining high grades. During my Senior year, I quit my job at Home Depot to focus more on my education, even though my supervisors wanted me to stay. I was an employee who always did the work that was required of me, and went above and beyond to do so. 
I took many courses while in university, incuding: Data Structures, Web Development, Cryptography, Programming Practicum, Discrete Structures, Software Development and Quality Assurance Techniques, Computer Operating Systems, Artificial Intelligence and Knowledge Acquisition, Database Systems Design, 
Machine Learning, and Algorithms. I have gained many skills from these classes, and completed many projects, which are summarized below. 

Projects: 

Asymmetric Encryption App
Completion Date: December 2024
Course: Cryptography
• Implemented advanced cryptographic algorithms including SHA3 (sponge and SHAKE functions) and an
elliptic-curve encryption scheme (ECIES) in a Java application.
• Utilizes SHA3 functions to compute hashes, authentication tags, and encrypt a user-specified file, as well as
decrypt the cryptogram and verify the MAC
• Utilizes ECIES functions to create public keys, encrypt a file under the public key sign it with Schnorr, as well
as to decrypt the file with a private key, and verify it.
• Tech Stack: Java, IntelliJ, and Github.


Bookkeeping Web App 
Completion Date: July 2024
Course: Web Development
• Full-stack web application for managing a book catalog (view/add/remove books and update ratings) with
secure user registration, login, and role-based access control.
• Collaborated in a team of 5 to design and document a RESTful API backend (Node.js, Express, PostgreSQL),
using Docker for containerization and tools like ApiDoc/Postman for API documentation and testing.
• Adapted to a new codebase by swapping projects with another team and developing a React/Next.js front-end
that consumed their API, demonstrating flexibility and teamwork to deliver the project successfully.
• My Role: Team Leader, created and managed backend repository, resolved merges and pull requests,
implemented POST for backend and frontend, communicated with other teams, fixed various bugs.
• Tech Stack: Typescript, React.js, Next.js, Docker, ApiDoc, Postgres, Postman, WebStorm.


Social Media Data Neural Network
Completion Date: July 2024
Course: Machine Learning
• Built and evaluated machine learning models (Naive Bayes, linear regression, and deep neural networks) to predict user demographics (age group and gender) from social media data.
• Achieved prediction accuracies of 64% (age group) and 82% (gender) using a neural network, improving significantly over the baseline accuracy (˜59%) and showcasing strong analytical and problem-solving skills.
• My Role: Project Leader, created baseline results, worked on the likes dataset, assisted other team members.
• Tech Stack: Python, scikit-learn, Keras, TensorFlow, Ubuntu VM, PyCharm.


Dungeon Crawler
Completion Date: December 2023
Course: Software Development and Quality Assurance Techniques
• Developed a Java Swing dungeon crawler game featuring randomly generated dungeons with monsters and collectible items, plus multiple playable heroes.
• Implemented game mechanics including a minimap, inventory system, turn-based combat, and save/load functionality, following an MVC architecture and other design patterns (Singleton, Factory) for clean, maintainable code.
• Created thorough JUnit testing for the entire back-end model.
• Created a UML diagram for the project, used Pivotal Tracker and Toggl to organize and delegate tasks, and to track the time spent on each task.
• My Role: Team Leader, created and maintained the repository, created most of the model, including recursive dungeon generation and the inventory system, and worked on the View-Controller.
• Tech Stack: Java, Swing, IntelliJ, GitHub, Pivotal Tracker, Toggl



 

## Resume:
Sviatoslav Rutkovskyi
Kent, Washington, 98030
/ne571-992-8193 rutkovsky2003@gmail.com LinkedIn: sviatoslav-rutkovskyi
PROFILE
Highly motivated software engineer looking to work my dream job, equipped with quick learning ability proven by
academic success. Recent graduate of the University of Washington, Tacoma, with a BS in CS and 3.9 GPA, who is
eager to learn. Able to work under pressure while maintaining personal and professional integrity and capable of
demonstrating leadership given my experience as a sergeant in the US Army Reserves. Software developer with
more than a year of SQL, Web-Dev, ML, and AI experience, and multiple years of Java, Python, and JavaScript
experience.
TECHNICAL SKILLS
Programming Languages:Python, Java, C, C#, JavaScript, SQL, R, HTML/CSS
Programming Skills:Version Control, Debugging, Unit Testing, Agile Development, Code Review
Developer Tools:VS Code, MySQL Workbench, R Studio, IntelliJ, PyCharm, WebStorm, Cursor
Technologies/Frameworks:GitHub, ReactJS, NodeJS, ExpressJS, Git, Docker, Postman
Cognitive Skills:Quick Learning Ability, Analytical Thinking, Strong Problem-Solving Skills
Soft Skills:Communication, Entrepreneurial Spirit, Leadership, Task Management
Spoken Languages:Native Speaker of Ukrainian, Fluent in English and Russian
Relevant Coursework:ML, AI, Data Structures, Algorithms, Databases, Web Development, Cryptography
PROJECTS
Asymmetric Encryption App| Github December 2024
• Implemented advanced cryptographic algorithms including SHA3 (sponge and SHAKE functions) and an
elliptic-curve encryption scheme (ECIES) in a Java application.
• Utilizes SHA3 functions to compute hashes, authentication tags, and encrypt a user-specified file, as well as
decrypt the cryptogram and verify the MAC
• Utilizes ECIES functions to create public keys, encrypt a file under the public key sign it with Schnorr, as well
as to decrypt the file with a private key, and verify it.
• Tech Stack: Java, IntelliJ, and Github.
Bookkeeping Web App| Github Backend Github Frontend June 2024
• Full-stack web application for managing a book catalog (view/add/remove books and update ratings) with
secure user registration, login, and role-based access control.
• Collaborated in a team of 5 to design and document a RESTful API backend (Node.js, Express, PostgreSQL),
using Docker for containerization and tools like ApiDoc/Postman for API documentation and testing.
• Adapted to a new codebase by swapping projects with another team and developing a React/Next.js front-end
that consumed their API, demonstrating flexibility and teamwork to deliver the project successfully.
• My Role: Team Leader, created and managed backend repository, resolved merges and pull requests,
implemented POST for backend and frontend, communicated with other teams, fixed various bugs.
• Tech Stack: Typescript, React.js, Next.js, Docker, ApiDoc, Postgres, Postman, WebStorm.
Social Media Data Neural Network June 2024
• Built and evaluated machine learning models (Naive Bayes, linear regression, and deep neural networks) to
predict user demographics (age group and gender) from social media data.
• Achieved prediction accuracies of 64% (age group) and 82% (gender) using a neural network, improving
significantly over the baseline accuracy ( ˜59%) and showcasing strong analytical and problem-solving skills.
• My Role: Project Leader, created baseline results, worked on the likes dataset, assisted other team members.
• Tech Stack: Python, scikit-learn, Keras, TensorFlow, Ubuntu VM, PyCharm.Dungeon Crawler December 2023
• Developed a Java Swing dungeon crawler game featuring randomly generated dungeons with monsters and
collectible items, plus multiple playable heroes.
• Implemented game mechanics including a minimap, inventory system, turn-based combat, and save/load
functionality, following an MVC architecture and other design patterns (Singleton, Factory) for clean,
maintainable code.
• Created thorough JUnit testing for the entire back-end model.
• Created a UML diagram for the project, used Pivotal Tracker and Toggl to organize and delegate tasks, and to
track the time spent on each task.
• My Role: Team Leader, created and maintained the repository, created most of the model, including recursive
dungeon generation and the inventory system, and worked on the View-Controller.
• Tech Stack: Java, Swing, IntelliJ, GitHub, Pivotal Tracker, Toggl
EXPERIENCE
US Army Reserve July 2021 – Present
Sergeant, Bridge Crew Member, 12C Marysville, Washington
• Demonstrated integrity, dedication, and mental fortitude through military training and duties, allowing me to
perform under pressure.
• Taught classes up to and exceeding army standards and provided mentorship to lower enlisted soldiers, setting
them up for success.
• Led and collaborated with team members in high-pressure environments, honing strong leadership and
teamwork skills that translate to managing projects and coordinating with colleagues in tech settings.
Home Depot July 2022 – January 2024
Sales Associate Kent, Washington
• Quickly learned new skills and honed my communication skills, rapidly increasing my knowledge in new areas
• Provided excellent customer service to a diverse customer base, in a timely and efficient manner.
• Maintained a positive team environment while consistently meeting sales and service targets.
EDUCATION
University of Washington September 2022 – March 2025
Bachelor of Science in Computer Science -CGPA - 3.9 Tacoma, Washington
Bellevue College September 2019 – June 2021
Running Start Bellevue, Washington
EXTRACURRICULAR
• Participated in UWT SEP 2023 Programming Contest
• Participated in UWT MAY 2024 Programming Contest

 ## Cover Letter Template:
Dear Hiring Team

When I saw that you were looking for a Software Developer, I was excited because its exactly the job I was looking for to showcase my skills in Python development and agentic AI to assist your cause in building a virtual tax assistant. As someone who is an excellent problem solver, and eager to learn new skills, as demonstrated by my outstanding academic career at University of Washington (3.9 GPA), I am ready to take on the challanges that this position might entail. 

During my time in university I have built many projects where I showed my profficiency with the tech stack you are working with, for example, I lead a team of 5 to create RESTful API for a bookkeeping web application, and a sleek React-based frontend for it. In a different project, I created various machine learning models using sklearn and TensorFlow which predicted user profile information, and was able to impove gender accuracy to 82% from a baseline of 59%. Additionaly, my time as a Sergeant in the US Army reserves gave me experince in working under pressure and producing results in dynamic envoronments, which I can translate to the job here. 

Finaly, I'm excited about Keeper's mission of transforming the future of finance as it closely alignes with my personal goal of making the world a better place. I wish you luck in your efforts, and hope that you will consider this application. I'm ready to take my skills to the next level with your team, and look forward hearing back from you. 
Thank you, 

Sviatoslav Rutkovskyi



        

## Previous cover letter rejected
You just tried to create a cover letter, but the quality control rejected your cover letter

        ## Your attempted cover letter:
Dear Hiring Team,

I am thrilled to apply for the Full Stack Engineer Co-Op Internship at Stably. As a recent graduate from the University of Washington, Tacoma, with a Bachelor of Science in Computer Science and a GPA of 3.9, I am eager to leverage my skills and knowledge in software development to contribute to your innovative team. My technical proficiency and problem-solving abilities, combined with my passion for creating positive change, align perfectly with the role at Stably.

Throughout my academic career, I have developed full-stack applications, demonstrating my expertise in both front-end and back-end development. Notably, I led a team to create a comprehensive bookkeeping web app with a React/Next.js frontend and a robust Node.js, Express, and PostgreSQL backend. This experience honed my skills in API design, Docker containerization, and collaboration in team settings. Additionally, my role as a sergeant in the US Army Reserves has equipped me with leadership and teamwork skills, enabling me to thrive in high-pressure environments and collaborate effectively.

Stably's mission to enhance financial systems resonates deeply with my goal of making a meaningful impact through technology. I admire your commitment to innovation in finance and am eager to contribute to your projects with dedication and enthusiasm. I am confident that my technical skills, coupled with my commitment to excellence, make me an ideal candidate for this internship. I look forward to the opportunity to contribute to Stably’s success and to grow professionally with your team.

Thank you for considering my application.

Sincerely,

Sviatoslav Rutkovskyi


        ## Reason for rejection:
This cover letter reads like a generic, AI-generated template rather than a tailored message to Stably. The opening is polite but lacks a strong hook about why you specifically want to work at Stably—mentioning their stablecoin technology or blockchain mission would show you’ve done your research. Your body paragraph highlights relevant projects, but you should connect those experiences directly to the skills and responsibilities listed in the internship posting (e.g. emphasize any front-end frameworks or DevOps tools Stably uses). The closing is courteous but could include a clear call to action and reiterate your enthusiasm for this particular role. To improve, personalize the letter by referencing Stably’s products or culture, add an engaging story or outcome from your past work, and demonstrate exactly how your background will advance their goals.


        
        

## Previous cover letter rejected
You just tried to create a cover letter, but the quality control rejected your cover letter

        ## Your attempted cover letter:
Dear Hiring Team,

I am eager to apply for the Full Stack Engineer Co-Op Internship at Stably. Having graduated from the University of Washington, Tacoma, with a Bachelor of Science in Computer Science and a 3.9 GPA, I am keen on utilizing my software development skills and technical proficiency to contribute to your innovative team at Stably, especially with your pioneering blockchain and stablecoin technologies.

In my academic journey, I had the opportunity to lead a team in building a full-stack bookkeeping web app, showcasing expertise in both front-end and back-end development using React/Next.js and Node.js, Express, and PostgreSQL respectively. This project refined my capacities in API design, Docker containerization, and seamless teamwork. Further, my experience as a sergeant in the US Army Reserves endowed me with the leadership and collaboration strengths that are crucial in dynamic environments, perfectly aligning with roles at Stably.

Stably's mission to enhance financial systems through blockchain and stablecoins resonates deeply with my aspirations to create impactful technological solutions. I am particularly impressed with your commitment to fostering financial inclusion through innovative technology. I am enthusiastic about the opportunity to bring my dedication and technical expertise to your team, contributing meaningfully to your groundbreaking financial projects. I look forward to becoming an integral part of your mission-driven efforts.

Thank you for considering my application.

Sincerely,

Sviatoslav Rutkovskyi


        ## Reason for rejection:
While this cover letter is professional and follows a basic structure, it reads as formulaic and bears hallmarks of AI generation rather than a personalized message. Key areas for improvement:

1. Opening Paragraph Lacks Engagement
   • You restate your degree and GPA but miss an opportunity to grab attention with a specific insight or story (e.g., a moment when you discovered blockchain’s potential).
   • Try beginning with a brief anecdote or achievement that directly ties to Stably’s work in stablecoins.

2. Overly Generic Language and Structure
   • Phrases like “keen on utilizing my software development skills” and “technical proficiency” are vague. Instead, name a concrete Stably product (for example, Stably USD) and describe how you would enhance it.
   • The three‐paragraph framework is good, but each should feel distinct. Paragraph two mixes your team leadership and Army experience; break these into their own bullet points or sentences for clarity and impact.

3. Insufficient Tailoring to the Internship Role
   • The letter mentions Docker and full‐stack work, but doesn’t connect to the Co-Op internship’s responsibilities (e.g., API integrations with blockchain networks or front-end data visualization for DeFi dashboards).
   • Reference a project or coursework that directly used smart contracts, Web3 libraries, or cryptographic methods, since Stably focuses on blockchain.

4. Add Quantifiable Achievements and Personal Touches
   • You mention leading a team and working under pressure in the Army, but give a brief metric or result (e.g., “trained 15 soldiers to exceed certification benchmarks by 20%”).
   • Include one or two sentences about your personal motivation for financial inclusion through stablecoins—this strengthens cultural fit.

5. Tone and Authenticity
   • Soften overly formal or robotic phrases. Write as yourself: use first‐person active verbs (“I built,” “I collaborated,” “I designed”).
   • Proofread for small typos (e.g., ensure consistent tense and punctuation).

Overall, this draft needs stronger personalization, more detail on how your experiences map to Stably’s Co-Op responsibilities, and a more compelling opening. Rewriting with specific references to Stably’s products, quantifiable outcomes, and a story‐driven start will make your cover letter far more engaging and authentic.


        
        

## Previous cover letter rejected
You just tried to create a cover letter, but the quality control rejected your cover letter

        ## Your attempted cover letter:
Dear Hiring Team,

I am writing to express my interest in the Full Stack Engineer Co-Op Internship at Stably. As a recent graduate from the University of Washington, Tacoma, with a Bachelor of Science in Computer Science and a 3.9 GPA, I am particularly drawn to Stably's pioneering work in blockchain and stablecoin technologies. My academic projects and experiences have equipped me with a solid foundation in software development, and I am excited about the opportunity to contribute to Stably's mission of enhancing financial systems and promoting financial inclusion.

During my academic career, I led a team to develop a comprehensive bookkeeping web app, employing a React/Next.js frontend and a Node.js, Express, and PostgreSQL backend. This project honed my skills in API design, Docker containerization, and collaborative development. Notably, I successfully implemented a RESTful API and adapted to a new codebase, demonstrating flexibility and teamwork. Additionally, my role as a sergeant in the US Army Reserves has instilled in me strong leadership and communication skills, enabling me to thrive in dynamic environments, which aligns well with the responsibilities at Stably.

Stably's commitment to technological innovation and financial inclusion through blockchain resonates deeply with my aspirations to create impactful solutions. I am particularly impressed by your work with stablecoins, such as Stably USD, and I am eager to apply my technical skills and dedication to contribute meaningfully to your groundbreaking projects. I look forward to the possibility of joining your mission-driven team and supporting Stably's efforts to transform financial systems.

Thank you for considering my application.

Sincerely,

Sviatoslav Rutkovskyi


        ## Reason for rejection:
This cover letter reads as a polished but generic AI-generated response rather than a personalized message to Stably. While it follows a three-paragraph structure and mentions relevant projects and leadership experience, the opening is standard (“I am writing to express my interest...”), and the language is formulaic. To improve: 

1. Strengthen the opening with a specific hook – perhaps a brief anecdote or direct reference to Stably’s recent product launch or technical challenge that resonated with you. 
2. Inject your genuine voice and passion: explain why blockchain and stablecoins matter to you personally and how your background uniquely positions you to contribute. 
3. Tie your skills more directly to Stably’s advertised tech stack or responsibilities (e.g., mention familiarity with their CI/CD pipeline, smart contract languages, or specific frameworks). 
4. Close with an active next step (“I would welcome the opportunity to discuss how my experience in X and Y can help Stably accelerate its roadmap”).

Personalizing these elements will make the letter more engaging, less templated, and clearly written by you rather than an AI.


        